## Algorithm all IC

In [1]:
# Import libraries
import pandas as pd
import requests
import re
import unidecode  # Este método quita las Ñ 
import unicodedata 
import math
import time


Obtaining test database codified

In [2]:
# Getting APIs
# Hacerlo con Año firma del contrato (menos) o año subida del SECOP (mas)?
import requests
url_secop = 'https://www.datos.gov.co/resource/c6dm-udt9.json'
p_invias = {'nombre_de_la_entidad': 'INSTITUTO NACIONAL DE VÍAS (INVIAS)', 
            '$limit': 10000, 
            'causal_de_otras_formas_de_contratacion_directa': 'Contratos Interadministrativos (Literal C)'}
r_invias = requests.get(url_secop, params = p_invias)

# To .json
d_invias = r_invias.json()
# To df
import pandas as pd
df_invias_all = pd.DataFrame(d_invias)

# Filter per year
df_invias_all[['anno_firma_del_contrato']] = df_invias_all[['anno_firma_del_contrato']].apply(pd.to_numeric)
df_invias = df_invias_all.loc[df_invias_all['anno_firma_del_contrato'] >= 2012]

Pequeña caracterizacion de DB de IC en INVIAS

In [4]:
# Hacer grafica de los años de los IC y de las filas que tienen este field en null
df_invias['anno_firma_del_contrato'].value_counts()

2013.0    1553
2012.0     844
2014.0     448
2015.0      71
2017.0      58
2016.0      52
Name: anno_firma_del_contrato, dtype: int64

Obtener información de los municipios

In [4]:
# Getting all municipios
i_razsoc = df_invias['nom_raz_social_contratista'].tolist()

i_mun = [ item for item in i_razsoc if 'MUNICIPIO' in item] + [ item for item in i_razsoc if 'DEPARTAMENTO' in item]
i_mun = [ item for item in i_mun if 'ADMINISTRATIVO' not in item]
i_mun = [ item for item in i_mun if 'AGENCIA' not in item]

dff_invias = df_invias[df_invias['nom_raz_social_contratista'].isin(i_mun)]
i_mun = dff_invias['nom_raz_social_contratista'].tolist()

# API con division politico-administrativa
url_deptos= 'https://www.datos.gov.co/resource/p95u-vi7k.json'
p_deptos = {'$limit': 2000}
r_deptos = requests.get(url_deptos, params = p_deptos)
r_deptos.status_code
r_deptos.url
# To .json
d_deptos = r_deptos.json()
# To df
df_deptos = pd.DataFrame(d_deptos)
# To upper
df_deptos['departamento'] = df_deptos['departamento'].str.upper()
df_deptos['municipio'] = df_deptos['municipio'].str.upper()
# Corregir casos particulares
df_deptos['municipio'] = [re.sub('EL CARMEN DE BOLÍVAR','CARMEN DE BOLÍVAR',item) for item in df_deptos['municipio']] 
df_deptos['municipio'] = [re.sub('EL CARMEN DE VIBORAL','CARMEN DE VIBORAL',item) for item in df_deptos['municipio']] 
df_deptos['municipio'] = [re.sub('PROVIDENCIA','PROVIDENCIA Y SANTA CATALINA',item) for item in df_deptos['municipio']] 
df_deptos['municipio'] = [re.sub('ESPINAL','EL ESPINAL',item) for item in df_deptos['municipio']] 
df_deptos['municipio'] = [re.sub('ITAGUI','ITAGÜÍ',item) for item in df_deptos['municipio']] 
df_deptos['municipio'] = [re.sub('TOLÚ VIEJO','TOLUVIEJO',item) for item in df_deptos['municipio']] 
df_deptos['municipio'] = [re.sub('TIMBIQUÍ','TIMBIQUI',item) for item in df_deptos['municipio']] 
df_deptos['municipio'] = [re.sub('CHIPATÁ','CHIPATA',item) for item in df_deptos['municipio']] 
df_deptos['municipio'] = [re.sub('SUSACON','SUSACÓN',item) for item in df_deptos['municipio']] 
df_deptos['municipio'] = [re.sub('TIMBÍO','TIMBIO',item) for item in df_deptos['municipio']] 
df_deptos['municipio'] = [re.sub('CURITÍ','CURITI',item) for item in df_deptos['municipio']] 
df_deptos['departamento'] = [re.sub('ARCHIPIÉLAGO DE SAN ANDRÉS, PROVIDENCIA Y SANTA CATALINA','SAN ANDRÉS PROVIDENCIA Y SANTA CATALINA',item) for item in df_deptos['departamento']] 



In [5]:
# STANDARIZE ALL LIST OF MUNICIPIOS

# Taken from: https://stackoverflow.com/questions/14153364/reorder-string-using-regular-expressions
def standarize_mun(mun):
    if "MUNICIPIO DE" not in mun:  
        mun = re.sub('MUNICIPIO','MUNICIPIO DE',mun)
    nmun = re.sub('DEPARTAMENTO DE|DEPARTAEMNTO DE| EN EL DEPARTAMENTO DE | EN EL DEPARTAMENTO DEL',' - ',mun)     #Caso: DEPARTAMENTO DE
    nmun = re.sub('(C/MARCA)',' - CUNDINAMARCA ', nmun)     #Caso: (C/MARCA)
    # Remove double spaces and punctuation (except -) https://stackoverflow.com/questions/1546226/simple-way-to-remove-multiple-spaces-in-a-string
    nmun = re.sub('\.','',nmun)     # https://stackoverflow.com/questions/265960/best-way-to-strip-punctuation-from-a-string-in-python
    nmun = re.sub(' +', ' ', nmun)
    nmun = re.sub('[(){}<>]', '', nmun)   # Parentesis
    r = re.compile('(^.*)(-)(.*$)')
    nmun = r.sub(r'\3'+ ' - ALCALDÍA '+ r'\1',nmun)
    nmun = nmun.lstrip()            # Remove spaces before beginning
    nmun = re.sub('\.','',nmun)
    return nmun 

def standarize_depto(mun):
    nmun = re.sub('GOBERNACION','',mun)     #Caso: DEPARTAMENTO DE
    nmun = re.sub('DEPARTAMENTO DEL','DEPARTAMENTO DE',nmun)     #Caso: DEPARTAMENTO DE
    nmun = re.sub('DEPARTAMENTO DE','GOBERNACIÓN -',nmun)     #Caso: DEPARTAMENTO DE
    # Remove double spaces and punctuation (except -) https://stackoverflow.com/questions/1546226/simple-way-to-remove-multiple-spaces-in-a-string
    nmun = re.sub('\.','',nmun)     # https://stackoverflow.com/questions/265960/best-way-to-strip-punctuation-from-a-string-in-python
    nmun = re.sub(' +', ' ', nmun)
    nmun = re.sub('[(){}<>]', '', nmun)   # Parentesis
    r = re.compile('(^.*)(-)(.*$)')
    nmun = r.sub(r'\3'+ ' - '+ r'\1',nmun)
    nmun = nmun.lstrip()            # Remove spaces before beginning
    nmun = re.sub('\.','',nmun)
    nmun = re.sub(' +', ' ', nmun)
    return nmun 

def strip_accents(s):
    return ''.join(c for c in unicodedata.normalize('NFD', s)
              if not unicodedata.name(c).endswith('ACCENT')) 

i_mun = [strip_accents(item) for item in i_mun] #SIN TILDES Y DEJA LA Ñ!!
# i_mun = [unidecode.unidecode(item) for item in i_mun] #SIN TILDES 
# i_mun = [unidecode.unidecode(item) for item in i_mun] #Añadir Ñ
# List with standarized municipios: TODO ver casos especiales
# i_mun = list(set(i_mun))
i_muns = []

# Corregir casos particulares
i_mun = [re.sub('MUNICIPIO DEL CARMEN DE BOLIVAR','MUNICIPIO DE CARMEN DE BOLIVAR',item) for item in i_mun] 
i_mun = [re.sub('MUNICIPIO DE EL CARMEN DE BOLIVAR','MUNICIPIO DE CARMEN DE BOLIVAR',item) for item in i_mun] 
i_mun = [re.sub('MUNICIPIO DEL CARMEN DE BOLIVAR','MUNICIPIO DE CARMEN DE BOLIVAR',item) for item in i_mun] 
i_mun = [re.sub('MUNICIPIO DE EL CARMEN DE BOLIVAR','MUNICIPIO DE CARMEN DE BOLIVAR',item) for item in i_mun] 
i_mun = [re.sub('SANTIAGO DE CALI','CALI',item) for item in i_mun] 
i_mun = [re.sub('SAN JOSE DE CUCUTA','CUCUTA',item) for item in i_mun] 
i_mun = [re.sub('MUNICIPIO EL CERRITO','MUNICIPIO DE EL CERRITO',item) for item in i_mun] 
i_mun = [re.sub('LE RETEN','EL RETEN',item) for item in i_mun] 
i_mun = [re.sub('PROVIDENCIA Y SANTA CATALINA ISLAS','PROVIDENCIA Y SANTA CATALINA',item) for item in i_mun] 
i_mun = [re.sub('SAN JUAN BAUTISTA DE GUACARI','GUACARI',item) for item in i_mun] 
i_mun = [re.sub('SUSACON','SUSACÓN',item) for item in i_mun] 


for item in i_mun:
    if 'MUNICIPIO' in item:
        i_muns.append(standarize_mun(item))
    else:
        i_muns.append(standarize_depto(item))

# Corregir errores particulares


       

#unaccented_string = unidecode.unidecode(accented_string)
deptos_t = list(set(df_deptos['departamento'].tolist()))  # https://stackoverflow.com/questions/7961363/removing-duplicates-in-lists
deptos_st = [strip_accents(item) for item in deptos_t]

mun_t = list(set(df_deptos['municipio'].tolist())) # CON TILDES
mun_st = [strip_accents(item) for item in mun_t] #SIN TILDES

# Estandarizar tildes deptos
for contd, depto in enumerate(deptos_st):
    for i, item in enumerate(i_muns):
        if depto in item:
            i_muns[i] = re.sub(depto,deptos_t[contd],item)
            
# Estandarizar tildes mun
for contm, mun in enumerate(mun_st):
    for i, item in enumerate(i_muns):
        if mun in item:
            i_muns[i] = re.sub(mun,mun_t[contm],item)
            
            
# Corregir casos particulares
i_muns = [re.sub('MANÍZALES','MANIZALES',item) for item in i_muns]  
i_muns = [re.sub('CUERQUIA','CUERQUÍA',item) for item in i_muns]  
i_muns = [re.sub('ZAPAYAN','ZAPAYÁN',item) for item in i_muns]     

# 1. Municipios que tienen el nombre del depto
for i, item in enumerate(i_muns):
    for contd, depto in enumerate(deptos_t):
        if '-' in item: break     # Si ya esta estendarizado
        item = item.lstrip()
        if 'CARMEN DE BOLÍVAR' in item: continue
        if depto in item:
            # https://stackoverflow.com/questions/30232344/insert-a-string-before-a-substring-of-a-string
            my_str = item
            substr = depto
            inserttxt = " - "
            idx = my_str.index(substr)
            i_muns[i] = my_str[:idx] + inserttxt + my_str[idx:]
            i_muns[i] = standarize_mun(i_muns[i])
            
    

# 2. Municipios que no tienen nombre de depto
for i, item in enumerate(i_muns):
    if '-' in item: continue
    string = i_muns[i].replace("MUNICIPIO DE ", "")  # remove the 8 from the string borders
    string = string.lstrip()
    for contm, mun in enumerate(df_deptos['municipio']):
        if mun == string:
            i_muns[i] = df_deptos['departamento'][contm] + ' - ' 'ALCALDÍA MUNICIPIO DE ' + string

i_muns = [s.rstrip() for s in i_muns]
i_muns = [re.sub('MUNICIPIO DEL GUAMO','TOLIMA - ALCALDÍA MUNICIPIO DE EL GUAMO',item) for item in i_muns] 

# Añadir columna nueva al DF
dff_invias = dff_invias.assign(nom_raz_soc_stand = i_muns)
i_muns = list(set(i_muns))

Cargar código para ASM

In [6]:
# ESTANDARIZACION DEL OBJETO A CONTRATAR
# Taken from: https://stackoverflow.com/questions/5541745/get-rid-of-stopwords-and-punctuation
from nltk.corpus import stopwords
import nltk
nltk.download('punkt')
nltk.download('stopwords')
def remove_stopwords(sentence, language):
    return [token for token in nltk.word_tokenize(sentence) if token.lower() not in stopwords.words(language)]

# Estandariza el detalle del objeto a contratar
# Taken from: https://stackoverflow.com/questions/14153364/reorder-string-using-regular-expressions
cachedStopWords = stopwords.words("spanish")
cachedStopWords = [x.upper() for x in cachedStopWords]
def standarize_obj(obj):
    nobj = obj.upper()
    nobj = ' '.join([word for word in nobj.split() if word not in cachedStopWords])
    nobj = unidecode.unidecode(nobj) # Sin tildes (y sin ñ)
    nobj = re.sub(r'[^\w\s]','',nobj)     # https://stackoverflow.com/questions/265960/best-way-to-strip-punctuation-from-a-string-in-python
    nobj = re.sub(' +', ' ', nobj)   
    return nobj

###### ACA EMPIEZA LA ITERACION GRANDE ###########
### INTENTO CON  TD-IDF
        ### https://bergvca.github.io/2017/10/14/super-fast-string-matching.html

# 1. Preparar funciones y librerias
import re

# Define la funcion ngrams que divide el string entre grupos de 3 para 
# que la comparación sea mucho mas rápida
def ngrams(string, n=3):
    string = re.sub(r'[,-./]|\sBD',r'', string)
    ngrams = zip(*[string[i:] for i in range(n)])
    return [''.join(ngram) for ngram in ngrams]

from sklearn.feature_extraction.text import TfidfVectorizer
import time
import numpy as np
from scipy.sparse import csr_matrix
import sparse_dot_topn.sparse_dot_topn as ct
import datetime


# En esta funcion se evalua la distancia entre los dos strings con cosine similarity
# @ntop: numero de coincidencias que se quieren tener por fila (por string)
def awesome_cossim_top(A, B, ntop, lower_bound=0):
    # force A and B as a CSR matrix.
    # If they have already been CSR, there is no overhead
    A = A.tocsr()
    B = B.tocsr()
    M, _ = A.shape
    _, N = B.shape
 
    idx_dtype = np.int32
 
    nnz_max = M*ntop
 
    indptr = np.zeros(M+1, dtype=idx_dtype)
    indices = np.zeros(nnz_max, dtype=idx_dtype)
    data = np.zeros(nnz_max, dtype=A.dtype)

    ct.sparse_dot_topn(
        M, N, np.asarray(A.indptr, dtype=idx_dtype),
        np.asarray(A.indices, dtype=idx_dtype),
        A.data,
        np.asarray(B.indptr, dtype=idx_dtype),
        np.asarray(B.indices, dtype=idx_dtype),
        B.data,
        ntop,
        lower_bound,
        indptr, indices, data)

    return csr_matrix((data,indices,indptr),shape=(M,N))

# ESTA FUNCION FUE MODIFICADA DE LA FUNCION ORIGINAL DE LA PAG WEB
    #En esta función se realiza un DF de la función awesome_cossim_top
    # solamente con los valores de imun
def get_matches_df(sparse_matrix, name_vector):
    non_zeros = sparse_matrix.nonzero()
    
    sparserows = non_zeros[0]
    sparsecols = non_zeros[1]
    
    nr_matches = sparsecols.size
    
    left_side = np.empty([nr_matches], dtype=object)
    right_side = np.empty([nr_matches], dtype=object)
    similairity = np.zeros(nr_matches)
    pos_left = np.zeros(nr_matches, dtype=np.int)
    pos_right = np.zeros(nr_matches, dtype=np.int)
    
    for index in range(0, nr_matches):
        left_side[index] = name_vector[len(name_vector)-1]
        right_side[index] = name_vector[sparsecols[index]]
        similairity[index] = sparse_matrix.data[index]
        pos_left[index] = sparserows[index]
        pos_right[index] = sparsecols[index]
    
    return pd.DataFrame({'left_side': left_side,
                          'right_side': right_side,
                           'similairity': similairity,
                            'pos_left': pos_left,
                            'pos_right': pos_right})


def get_matches_df1(sparse_matrix, name_vector):
    non_zeros = sparse_matrix.nonzero()
    
    sparserows = non_zeros[0]
    sparsecols = non_zeros[1]
    
    nr_matches = sparsecols.size
    
    left_side = np.empty([nr_matches], dtype=object)
    right_side = np.empty([nr_matches], dtype=object)
    similairity = np.zeros(nr_matches)
    pos_left = np.zeros(nr_matches, dtype=np.int)
    pos_right = np.zeros(nr_matches, dtype=np.int)
    
    for index in range(0, nr_matches):
        left_side[index] = name_vector[sparserows[index]]
        right_side[index] = name_vector[sparsecols[index]]
        similairity[index] = sparse_matrix.data[index]
        pos_left[index] = sparserows[index]
        pos_right[index] = sparsecols[index]
    
    return pd.DataFrame({'left_side': left_side,
                          'right_side': right_side,
                           'similairity': similairity,
                            'pos_left': pos_left,
                            'pos_right': pos_right})


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\melis\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\melis\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


ASM ALGORITHM - ALL

In [7]:
# 2. Ejecutar el código
fallos = []
cinter = pd.DataFrame()
fallidos = 0 
cont = 0
t_proceso = 10000000
threshold = 0.8
tol_imun = 0.9
contador = 1 
cont1 = 0
continv = 0
ti_dfinvias = []
ti_api = []
ti_token = []
ti_matrix = []
ti_printdb = []
ti_append = []
ti_fuzzy = []
ti_t = []
sorter = ['Liquidado', 'Terminado Sin Liquidar', 'Celebrado', 'Adjudicado', 'Convocado']

start = time.time()
for i, item in enumerate(i_muns):
    t_b = time.time()
    print ("Iteracion")
    print (i)
   # Subsetting INVIAS df
    df_imun = dff_invias.loc[dff_invias['nom_raz_soc_stand'] == item]
    df_imun = df_imun[df_imun['estado_del_proceso'].isin(sorter)]
    df_imun = df_imun.reset_index(drop=True)  
    df_imun['duplicated'] = np.zeros(len(df_imun), dtype=np.int)
    df_imun['final'] = np.ones(len(df_imun), dtype=bool)
    # Crear vector de detalles de objeto a contratar
    l_imun = df_imun['detalle_del_objeto_a_contratar']
    l_imun = list(map(str, l_imun))
    l_imun = pd.Series(l_imun)
    # Ejecutar matriz de comparación
    vectorizer1 = TfidfVectorizer(min_df=1, analyzer=ngrams)
    tf_idf_matrix1 = vectorizer1.fit_transform(l_imun)         #transforma a matriz
    matches1 = awesome_cossim_top(tf_idf_matrix1, tf_idf_matrix1.transpose(), 4, 0.0)   #max se repite 4 veces un contrato
    matches_df1 = get_matches_df1(matches1, l_imun)
    temp = -1
    for indexf, fila in matches_df1.iterrows():
        if temp < fila['pos_left']:
            temp = fila['pos_left']
            continv = continv + 1
        if fila['pos_left'] == fila['pos_right']: 
            continue
        elif fila['similairity'] > tol_imun:
            df_imun['duplicated'].loc[fila['pos_right']] = continv
            df_imun['duplicated'].loc[fila['pos_left']] = continv
        else:
            df_imun['duplicated'].loc[fila['pos_left']] = continv
        temp = fila['pos_left']
    # https://stackoverflow.com/questions/23482668/sorting-by-a-custom-list-in-pandas
    df_imun.estado_del_proceso = df_imun.estado_del_proceso.astype("category")
    df_imun.estado_del_proceso.cat.set_categories(sorter, inplace=True)
    for contador in range (1,max(df_imun['duplicated'])+1):
        if len(df_imun[df_imun['duplicated'] == contador]) > 1:    
            tdf = df_imun[df_imun['duplicated'] == contador]
            tdf.sort_values(["estado_del_proceso"], inplace = True)
            c = 0 
            for index, row in tdf.iterrows():
                if c == 0: 
                    df_imun['final'].loc[index] = True 
                    c = c+1
                else: df_imun['final'].loc[index] = False
        else:
            df_imun['final'][df_imun['duplicated'] == contador] = True 
    
    t_invias = time.time()
    tinvias= t_invias - t_b
    ti_dfinvias.append(tinvias)
    
    # Getting API from municipio 
    p_mun = {'nombre_de_la_entidad': i_muns[i], '$limit': 1000000}
    r_mun = requests.get(url_secop, params = p_mun)
    d_mun = r_mun.json()   # To .json
    df_mun = pd.DataFrame(d_mun)    # To df
    if df_mun.empty:
        fallos.append(i_muns[i])
        print (i_muns[i])
        fallidos = fallidos + 1
        print ("DB MUN Empty")
        print("-----------------------")
        ti_api.append(0)
        ti_fuzzy.append(0)
        ti_t.append(time.time()-t_b)
        continue
    df_mun = df_mun[df_mun['estado_del_proceso'].isin(sorter)]
    if df_mun.empty: continue
    df_mun[['cuantia_proceso']] = df_mun[['cuantia_proceso']].apply(pd.to_numeric)
    df_mun[['anno_firma_del_contrato']] = df_mun[['anno_firma_del_contrato']].apply(pd.to_numeric)
    df_mun = df_mun.reset_index(drop=True)
    print (i_muns[i])
    
    df_imun[['cuantia_proceso']] = df_imun[['cuantia_proceso']].apply(pd.to_numeric)
    df_imun[['anno_firma_del_contrato']] = df_imun[['anno_firma_del_contrato']].apply(pd.to_numeric)
    df_imun = df_imun.reset_index(drop=True) 
    
    t_api = time.time()
    print('Time API: ')
    tapi= t_api - t_invias
    ti_api.append(tapi)
    print(tapi)
    
    
    # TD-IDF String Matching
    for index, row in df_imun.iterrows():    
        # No iterar en los que tienen falso
        if row['final'] == False: 
            df1 = row.to_frame().T
            df1.index = [cont]
            cinter = cinter.append(df1, sort = False)
            cont = cont + 1
            continue
            
        c_proceso = row['cuantia_proceso']
        if c_proceso == 0: continue
        # Subset reglas (temporalidad, valores)
        anno_imun = row['anno_firma_del_contrato']
        df_mun_f = df_mun[(df_mun.anno_firma_del_contrato >= anno_imun)|(df_mun['anno_firma_del_contrato'].isnull())]
        df_mun_f = df_mun_f.reset_index(drop=True) 
         
        
        # Crear vector de detalles de objeto a contratar
        b = df_mun_f['detalle_del_objeto_a_contratar']
        b = list(map(str, b))
        b = list(map(standarize_obj, b))
        b = pd.Series(b)
        a = pd.Series(standarize_obj(row['detalle_del_objeto_a_contratar']))
        objeto = list(b) + list(a)
        # Ejecutar matriz de comparación
        vectorizer = TfidfVectorizer(min_df=1, analyzer=ngrams)
        tf_idf_matrixO = vectorizer.fit_transform(objeto)         #transforma a matriz
        
        t_token = time.time()
        ttoken= t_token - t_api
        ti_token.append(ttoken)
        
        matches = awesome_cossim_top(tf_idf_matrixO, tf_idf_matrixO.transpose(), 3, 0)
        
        t_matrix = time.time()
        tmatrix= t_matrix - t_token
        ti_matrix.append(tmatrix)
        
        #tf_idf_matrixA = vectorizer.fit_transform(a)         #transforma a matriz
        #tf_idf_matrixB = vectorizer.fit_transform(b)
        #matches = awesome_cossim_top(tf_idf_matrixA, tf_idf_matrixB.transpose(), 10, 0)
        #Se obtiene el df con la informacion de los puntajes mas altos
        matches_df = get_matches_df1(matches, objeto) 
        matches_df = matches_df.loc[matches_df['left_side'] == a[0]]
        matches_df = matches_df.loc[matches_df['pos_left'] == (len(objeto)-1)]
        
        t_db = time.time()
        tdb= t_db - t_matrix
        ti_printdb.append(tdb)     
        
        for index_ci, fila_ci in matches_df.iterrows():
            if fila_ci['pos_left'] == fila_ci['pos_right']: continue    #Si es él mismo no cuenta
            score = fila_ci['similairity']
            df1 = row.to_frame().T
            df2 = df_mun_f.loc[fila_ci['pos_right']].to_frame().T
            df1.index = [cont]
            df2.index = [cont]
            df2.columns = [str(col) + '_m' for col in df2.columns]
            result = pd.concat([df1, df2], axis=1, join = 'inner', sort = True)       #Concatena la info de SECOP
            cinter = cinter.append(result, sort = False)
            cinter.at[cinter.index[cont], 'score'] = score
            if score > threshold: cinter.at[cinter.index[cont], 'valido'] = True
            else: cinter.at[cinter.index[cont], 'valido'] = False
            cont = cont + 1
        t_app = time.time()
        tapp = t_app - t_db
        ti_append.append(tapp)
                
    t_fuzzy = time.time()
    print('Time Fuzzy: ')
    tfuzzy=t_fuzzy-t_api
    ti_fuzzy.append(tfuzzy)
    print(tfuzzy)
    
    print('TOTAL TIME: ')
    tt = t_fuzzy - t_b
    ti_t.append(tt)
    print(tt)
    print("-----------------------")

end = time.time()
total = end - start

Iteracion
0


C:\Users\melis\Anaconda3\lib\site-packages\pandas\core\indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
C:\Users\melis\Anaconda3\lib\site-packages\ipykernel_launcher.py:69: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


CÓRDOBA - GOBERNACIÓN
Time API: 
6.87278151512146
Time Fuzzy: 
43.898738861083984
TOTAL TIME: 
52.86988687515259
-----------------------
Iteracion
1
BOLÍVAR - ALCALDÍA MUNICIPIO DE MONTECRISTO
Time API: 
1.862623929977417
Time Fuzzy: 
0.2324047088623047
TOTAL TIME: 
2.1010539531707764
-----------------------
Iteracion
2
BOLÍVAR - ALCALDÍA MUNICIPIO DE SANTA ROSA
DB MUN Empty
-----------------------
Iteracion
3
CAQUETÁ - ALCALDÍA MUNICIPIO DE ALBÁNIA
DB MUN Empty
-----------------------
Iteracion
4
BOYACÁ - ALCALDÍA MUNICIPIO DE PAJARITO
Time API: 
2.3079822063446045
Time Fuzzy: 
0.46779537200927734
TOTAL TIME: 
3.7883501052856445
-----------------------
Iteracion
5
BOLÍVAR - ALCALDÍA MUNICIPIO DE REGIDOR
Time API: 
1.1803491115570068
Time Fuzzy: 
0.11979103088378906
TOTAL TIME: 
1.3051280975341797
-----------------------
Iteracion
6
GUAJIRA - ALCALDÍA MUNICIPIO DE FONSECA
DB MUN Empty
-----------------------
Iteracion
7
SANTANDER - ALCALDÍA MUNICIPIO DE MOLAGAVITA
Time API: 
1.44695091

Time Fuzzy: 
1.6223676204681396
TOTAL TIME: 
11.218484163284302
-----------------------
Iteracion
55
ARAUCA - ALCALDÍA MUNICIPIO DE FORTUL
Time API: 
1.6965653896331787
Time Fuzzy: 
0.44603753089904785
TOTAL TIME: 
2.147588014602661
-----------------------
Iteracion
56
CUNDINAMARCA - ALCALDÍA MUNICIPIO DE CHOACHÍ
Time API: 
2.116910219192505
Time Fuzzy: 
2.287797689437866
TOTAL TIME: 
11.273493528366089
-----------------------
Iteracion
57
ANTIOQUIA - ALCALDÍA MUNICIPIO DE ITAGUI
DB MUN Empty
-----------------------
Iteracion
58
ANTIOQUIA - ALCALDÍA MUNICIPIO DE REMEDIOS
Time API: 
1.9780664443969727
Time Fuzzy: 
1.3633873462677002
TOTAL TIME: 
9.080705165863037
-----------------------
Iteracion
59
NORTE DE SANTANDER - ALCALDÍA MUNICIPIO DE EL CARMEN
Time API: 
2.0892608165740967
Time Fuzzy: 
1.1659152507781982
TOTAL TIME: 
8.694561243057251
-----------------------
Iteracion
60
SANTANDER - ALCALDÍA MUNICIPIO DE ONZAGA
Time API: 
1.151662826538086
Time Fuzzy: 
0.18656110763549805
TOTAL 

CUNDINAMARCA - ALCALDÍA MUNICIPIO DE UBAQUE
Time API: 
1.3664371967315674
Time Fuzzy: 
3.5845770835876465
TOTAL TIME: 
15.715040922164917
-----------------------
Iteracion
108
SANTANDER - ALCALDÍA MUNICIPIO DE GUAPOTÁ
DB MUN Empty
-----------------------
Iteracion
109
CALDAS - ALCALDÍA MUNICIPIO DE MANZANARES
Time API: 
1.401047706604004
Time Fuzzy: 
2.1163012981414795
TOTAL TIME: 
13.765105724334717
-----------------------
Iteracion
110
HUILA - ALCALDÍA MUNICIPIO DE SAN AGUSTÍN
Time API: 
1.1799895763397217
Time Fuzzy: 
0.34012579917907715
TOTAL TIME: 
1.525101661682129
-----------------------
Iteracion
111
GUAVIARE - ALCALDÍA MUNICIPIO DE MIRAFLORES
Time API: 
1.1986401081085205
Time Fuzzy: 
0.35616612434387207
TOTAL TIME: 
1.5597572326660156
-----------------------
Iteracion
112
- ALCALDÍA MUNICIPIO DE CIÉNAGA -MAGDALENA
DB MUN Empty
-----------------------
Iteracion
113
NORTE DE SANTANDER - ALCALDÍA MUNICIPIO DE DURANIA
Time API: 
1.2691266536712646
Time Fuzzy: 
0.6722373962402344


...

Time Fuzzy: 
2.806735038757324
TOTAL TIME: 
80.87886428833008
-----------------------
Iteracion
845
RISARALDA - ALCALDÍA MUNICIPIO DE PEREIRA
Time API: 
7.2524144649505615
Time Fuzzy: 
79.00474643707275
TOTAL TIME: 
161.80581426620483
-----------------------
Iteracion
846
CALDAS - MUNCIPIO DE MARMATO DEL GOBERNACIÓN
DB MUN Empty
-----------------------
Iteracion
847
NARIÑO - ALCALDÍA MUNICIPIO DE OSPINA DEL
DB MUN Empty
-----------------------
Iteracion
848
NORTE DE SANTANDER - ALCALDÍA MUNICIPIO DE SARDINATA
Time API: 
1.2047533988952637
Time Fuzzy: 
1.6136462688446045
TOTAL TIME: 
80.95542001724243
-----------------------
Iteracion
849
SANTANDER - ALCALDÍA MUNICIPIO DE LA BELLEZA
Time API: 
1.124988317489624
Time Fuzzy: 
0.7998638153076172
TOTAL TIME: 
77.96791076660156
-----------------------
Iteracion
850
CAUCA - ALCALDÍA MUNICIPIO DE TORIBIO
DB MUN Empty
-----------------------
Iteracion
851
CUNDINAMARCA - ALCALDÍA MUNICIPIO DE SIBATÉ
Time API: 
1.4541435241699219
Time Fuzzy: 
2.9

META - ALCALDÍA MUNICIPIO DE SAN JUANITO
Time API: 
0.9751987457275391
Time Fuzzy: 
0.18849706649780273
TOTAL TIME: 
1.170677661895752
-----------------------
Iteracion
900
BOYACÁ - ALCALDÍA MUNICIPIO DE SABOYÁ
Time API: 
1.0871715545654297
Time Fuzzy: 
0.316251277923584
TOTAL TIME: 
1.410405158996582
-----------------------
Iteracion
901
ATLÁNTICO - ALCALDÍA MUNICIPIO DE LURUACO
Time API: 
1.138514757156372
Time Fuzzy: 
2.0784454345703125
TOTAL TIME: 
84.84301805496216
-----------------------
Iteracion
902
ANTIOQUIA - ALCALDÍA MUNICIPIO DE SAN RAFAEL
Time API: 
1.4003283977508545
Time Fuzzy: 
2.0934040546417236
TOTAL TIME: 
83.67394161224365
-----------------------
Iteracion
903
VICHADA - ALCALDÍA MUNICIPIO DE PUERTO CARREÑO
Time API: 
1.285139560699463
Time Fuzzy: 
0.6133604049682617
TOTAL TIME: 
1.9044840335845947
-----------------------
Iteracion
904
META - ALCALDÍA MUNICIPIO DE EL CALVARIO
Time API: 
1.0448482036590576
Time Fuzzy: 
0.20345640182495117
TOTAL TIME: 
1.25428843498229

CÓRDOBA - ALCALDÍA MUNICIPIO DE SAN JOSÉ DE URÉ
DB MUN Empty
-----------------------
Iteracion
952
LA GUAJIRA - ALCALDÍA MUNICIPIO DE HATONUEVO
Time API: 
0.9040572643280029
Time Fuzzy: 
0.21836423873901367
TOTAL TIME: 
1.1274096965789795
-----------------------
Iteracion
953
ANTIOQUIA - ALCALDÍA MUNICIPIO DE LA UNIÓN
Time API: 
1.4483819007873535
Time Fuzzy: 
1.4800751209259033
TOTAL TIME: 
87.27071595191956
-----------------------
Iteracion
954
CHOCÓ - ALCALDÍA MUNICIPIO DE ALTO DE BAUDO
DB MUN Empty
-----------------------
Iteracion
955
ATLÁNTICO - ALCALDÍA MUNICIPIO DE JUAN DE ACOSTA
Time API: 
0.7222387790679932
Time Fuzzy: 
0.672156810760498
TOTAL TIME: 
86.27864527702332
-----------------------
Iteracion
956
CAUCA - GOBERNACIÓN
Time API: 
3.107314109802246
Time Fuzzy: 
16.97873592376709
TOTAL TIME: 
104.89842796325684
-----------------------
Iteracion
957
BOYACÁ - ALCALDÍA MUNICIPIO DE MUZO
Time API: 
1.1900784969329834
Time Fuzzy: 
1.6477296352386475
TOTAL TIME: 
89.61476850509

CUNDINAMARCA - ALCALDÍA MUNICIPIO DE GIRARDOT
Time API: 
2.124145030975342
Time Fuzzy: 
9.502737760543823
TOTAL TIME: 
101.54501152038574
-----------------------
Iteracion
1006
VAUPÉS - ALCALDÍA MUNICIPIO DE CARURU
Time API: 
1.8942792415618896
Time Fuzzy: 
0.4019639492034912
TOTAL TIME: 
2.3032054901123047
-----------------------
Iteracion
1007
CALDAS - ALCALDÍA MUNICIPIO DE VITERBO
Time API: 
1.3151826858520508
Time Fuzzy: 
0.9684107303619385
TOTAL TIME: 
92.19399118423462
-----------------------
Iteracion
1008
ANTIOQUIA - ALCALDÍA MUNICIPIO DE SEGOVIA
Time API: 
1.353198766708374
Time Fuzzy: 
0.6123616695404053
TOTAL TIME: 
1.972541332244873
-----------------------
Iteracion
1009
L CAQUETÁ - ALCALDÍA MUNICIPIO DE MILÁN
DB MUN Empty
-----------------------
Iteracion
1010
HUILA - ALCALDÍA MUNICIPIO DE SANTA MARÍA
Time API: 
1.1005544662475586
Time Fuzzy: 
0.2862410545349121
TOTAL TIME: 
1.393775463104248
-----------------------
Iteracion
1011
CUNDINAMARCA - ALCALDÍA MUNICIPIO DE SASAI

C:\Users\melis\Anaconda3\lib\site-packages\ipykernel_launcher.py:61: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [2]:
total

NameError: name 'total' is not defined

Despues del algoritmo

In [1]:
total

NameError: name 'total' is not defined

In [11]:
cinterbu = cinter

df_tiempos1 = pd.DataFrame({'dfinvias':ti_dfinvias,
                           'api':ti_api,
                          'fuzzy':ti_fuzzy,
                          'total':ti_t})

df_tiempos2 = pd.DataFrame({'token':ti_token,
                          'matrix':ti_matrix,
                          'printdb':ti_printdb,
                          'append':ti_append})

In [12]:
cinter.to_csv("CI_INVIAS_all.csv", sep='\t', decimal = ',', encoding='latin1')


In [13]:
df_tiempos1.to_csv("t_INVIAS1_all.csv", sep='\t', decimal = ',', encoding='latin1')
df_tiempos2.to_csv("t_INVIAS2_all.csv", sep='\t', decimal = ',', encoding='latin1')